In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [8]:
# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Data augmentation for training data
training_data_generator = ImageDataGenerator(
    rotation_range = 18,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

# Only rescaling for test data
testing_data_generator = ImageDataGenerator()

# Create data generators
train_generator = training_data_generator.flow(X_train , y_train , batch_size=32)
test_generator = testing_data_generator.flow(X_test , y_test , batch_size=32)


In [9]:

model = Sequential([
    Conv2D(32 , (3, 3) , activation = "relu" , input_shape = (32, 32, 3)),
    Conv2D(64 , (3, 3) , activation = "relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128 , (3, 3) , activation = "relu"),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    BatchNormalization(),
    # Added more convolutional layers with dimension management
    Conv2D(128 , (3, 3) , activation = "relu" , padding = "same"),
    MaxPooling2D((2, 2) , padding = "same"),
    Dropout(0.4),
    BatchNormalization(),
    Flatten(),
    Dense(512 , activation = "relu"),
    Dropout(0.5),
    BatchNormalization(),
    Dense(10 , activation='softmax')
])


In [10]:

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
history = model.fit(train_generator , epochs = 90 , validation_data = (X_test , y_test))

Epoch 1/90
1563/1563 [==============================] - 43s 26ms/step - loss: 2.2902 - accuracy: 0.2474 - val_loss: 1.5671 - val_accuracy: 0.4279
Epoch 2/90
1563/1563 [==============================] - 41s 26ms/step - loss: 1.8838 - accuracy: 0.3349 - val_loss: 1.4448 - val_accuracy: 0.4742
Epoch 3/90
1563/1563 [==============================] - 41s 26ms/step - loss: 1.6964 - accuracy: 0.3903 - val_loss: 1.3956 - val_accuracy: 0.4898
Epoch 4/90
1563/1563 [==============================] - 40s 25ms/step - loss: 1.5565 - accuracy: 0.4402 - val_loss: 1.2568 - val_accuracy: 0.5450
Epoch 5/90
1563/1563 [==============================] - 42s 27ms/step - loss: 1.4483 - accuracy: 0.4792 - val_loss: 1.2538 - val_accuracy: 0.5508
Epoch 6/90
1563/1563 [==============================] - 43s 27ms/step - loss: 1.3653 - accuracy: 0.5156 - val_loss: 1.1203 - val_accuracy: 0.5999
Epoch 7/90
1563/1563 [==============================] - 43s 28ms/step - loss: 1.2978 - accuracy: 0.5384 - val_loss: 1.0638 -

In [12]:
model.save("image_classification_model.keras")


In [18]:
import numpy as np
import sklearn.metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_test_pred_labels = np.argmax(y_test_pred, axis=1)

# Evaluating on training data
train_accuracy = accuracy_score(np.argmax(y_train, axis=1), y_train_pred_labels)
train_precision = precision_score(np.argmax(y_train, axis=1), y_train_pred_labels, average='weighted')
train_recall = recall_score(np.argmax(y_train, axis=1), y_train_pred_labels, average='weighted')
train_f1 = f1_score(np.argmax(y_train, axis=1), y_train_pred_labels, average='weighted')

# Evaluating on test data
test_accuracy = accuracy_score(np.argmax(y_test, axis=1), y_test_pred_labels)
test_precision = precision_score(np.argmax(y_test, axis=1), y_test_pred_labels, average='weighted')
test_recall = recall_score(np.argmax(y_test, axis=1), y_test_pred_labels, average='weighted')
test_f1 = f1_score(np.argmax(y_test, axis=1), y_test_pred_labels, average='weighted')

# Printing the results
print(f"Training accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")
print(f"Training Precision: {train_precision}")
print(f"Training Recall: {train_recall}")
print(f"Training F1 Score: {train_f1}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")



313/313 [==============================] - 1s 2ms/step
Training accuracy: 0.83996
Test accuracy: 0.8096
Training Precision: 0.8560960679719812
Training Recall: 0.83996
Training F1 Score: 0.8386385421175236
Test Precision: 0.829486421011024
Test Recall: 0.8096
Test F1 Score: 0.8070543413265651


In [19]:
!pip install streamlit pyngrok

# Write the Streamlit app to a file
with open("streamlit.py" , "w") as f:
    f.write('''
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the trained model
try:
    model = load_model('image_classification_model.keras')
    st.write("Model has been loaded successfully.")
except OSError as e:
    st.write(f"Error loading the model: {e}")

# Define a function to preprocess the image
def preprocess_image(image):
    image = image.convert("RGB")  # Convert image to RGB
    size = (32, 32)  # Resize the image to the required input size of the model
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image = np.asarray(image)
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)
    return image

# Streamlit interface
st.title("Image Classification with Convolutional Nueral Networks")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    st.write("")
    st.write("Classifying your image")

    image = preprocess_image(image)

    predictions = model.predict(image)
    class_idx = np.argmax(predictions)

    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    class_name = class_names[class_idx]
    st.write(f"Predicted class: {class_name}")
''')

# Set up ngrok authtoken
!ngrok authtoken "2iak2kW5UoWTXIXYCHuwZGuvsLQ_2rHVLPosiGXgj6LeV1VeZ"

# Run the Streamlit app
get_ipython().system_raw("streamlit run streamlit.py &")

# Start ngrok and create a public URL
from pyngrok import ngrok

# Create tunnel to the Streamlit port 8501
public_url = ngrok.connect(addr = "8501" , bind_tls = True)
print(public_url)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://2198-34-87-84-133.ngrok-free.app" -> "http://localhost:8501"
